In [2]:
import io
import os.path
import re
import json
import numpy as np
import pandas as pd
from multiprocessing import Pool, Process, Manager
# Plotting tools
import spacy
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
from joblib import Parallel, delayed


In [26]:
df.iloc[19970]['versions']

"[{'version': 'v1', 'created': 'Thu, 16 Aug 2007 01:10:36 GMT'}]"

In [3]:
df = pd.read_csv('arxiv.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df_astro = df[df.categories == 'astro-ph']

In [13]:
df_astro['v1_time'] = df_astro["versions"].str.split('\'', expand = True)[7].values

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
df_astro['time']

8         2007-04-02 19:41:34
16        2007-03-31 07:38:48
22        2007-03-31 11:42:13
58        2007-03-31 22:16:50
79        2007-04-01 10:20:37
93        2007-04-02 18:30:16
127       2007-04-02 06:22:08
132       2007-04-02 06:51:10
138       2007-04-02 08:19:37
154       2007-04-02 09:38:00
155       2007-04-02 12:02:40
159       2007-04-02 10:25:01
167       2007-04-02 11:20:03
170       2007-04-02 13:45:22
183       2007-04-02 13:34:32
186       2007-04-02 13:31:00
191       2007-04-02 13:40:25
202       2007-04-02 14:24:07
204       2007-04-02 14:28:29
208       2007-04-02 14:58:36
218       2007-04-02 15:45:44
221       2007-04-02 15:52:54
222       2007-04-02 15:53:03
224       2007-04-02 16:13:32
225       2007-04-02 17:50:23
229       2007-04-02 16:42:38
233       2007-04-02 16:49:34
236       2007-04-02 17:14:27
237       2007-04-02 17:14:48
241       2007-04-02 17:26:49
                  ...        
1460772   1999-12-28 21:23:19
1460773   1999-12-28 22:15:17
1460774   

In [17]:
# df_astro['v1_time']

In [18]:
df_astro['time'] = pd.to_datetime(df_astro['v1_time'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
print(df_astro['time'].min(), df_astro['time'].max())

1992-04-13 18:20:01 2008-12-30 20:38:39


In [ ]:
df.to_csv('arxiv.csv')

In [ ]:
df_astro = df_astro.reset_index()
df_astro.to_csv('astro-ph.csv')



In [ ]:
data = pd.read_csv('astro-ph.csv')

In [ ]:
data['v1_time'] = ''

In [ ]:
# # def split_datetime(data):
# #     for i, row in enumerate(data['versions']):
# #         data['v1_time'].iloc[i] = data['versions'][i].split('\'')[-2]
# #     return data

# dt = [] 

# def split_datetime(data):
#     return data['versions'][i].split('\'')[-2]

# %timeit
# dt = Parallel(n_jobs=8)(delayed(split_datetime)(data) for i, row in enumerate(data['versions']))

In [ ]:
data['v1_time'] = data["versions"].str.split('\'', expand = True)[7].values

In [ ]:
# temp[7]

In [ ]:
data['time'].min()

In [ ]:
data['time'] = pd.to_datetime(data['v1_time'])

In [ ]:
data.to_csv('astro-ph.csv')

## Select a subset of data from 2010 to 2020

In [29]:
start_date = '2003-01-01'
end_date = '2008-11-30'

In [30]:
mask = (df_astro['time'] > start_date) & (df_astro['time'] <= end_date)
df_10 = df_astro.loc[mask]
df_10

,Unnamed: 0,id,authors,title,comments,journal-ref,doi,categories,abstract,versions,v1_time,time
8,8,704.001,"Paul Harvey, Bruno Merin, Tracy L. Huard, Luis...","The Spitzer c2d Survey of Large, Nearby, Inste...",NaN,"Astrophys.J.663:1149-1173,2007",10.1086/518646,astro-ph,We discuss the results from the combined IRA...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","Mon, 2 Apr 2007 19:41:34 GMT",2007-04-02 19:41:34
16,16,704.002,"Nceba Mhlahlo, David H. Buckley, Vikram S. Dhi...",Spectroscopic Observations of the Intermediate...,"10 pages, 11 figures (figures 3, 4, 7 and 8 at...","Mon.Not.Roy.Astron.Soc.378:211-220,2007",10.1111/j.1365-2966.2007.11762.x,astro-ph,Results from spectroscopic observations of t...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...","Sat, 31 Mar 2007 07:38:48 GMT",2007-03-31 07:38:48
22,22,704.002,"M. A. Loukitcheva, S. K. Solanki and S. White",ALMA as the ideal probe of the solar chromosphere,"4 pages, 2 figures, to appear in the proceedin...","Astrophys.Space Sci.313:197-200,2008",10.1007/s10509-007-9626-1,astro-ph,"The very nature of the solar chromosphere, i...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...","Sat, 31 Mar 2007 11:42:13 GMT",2007-03-31 11:42:13
58,58,704.006,"T. G. Beatty, J. M. Fernandez, D. W. Latham, G...",The Mass and Radius of the Unseen M-Dwarf Comp...,"18 pages, 9 tables and 6 figures; accepted by ...","Astrophys.J.663:573-582,2007",10.1086/518413,astro-ph,We derive masses and radii for both componen...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...","Sat, 31 Mar 2007 22:16:50 GMT",2007-03-31 22:16:50
79,79,704.008,Dean E. McLaughlin and S. Michael Fall,Shaping the Globular Cluster Mass Function by ...,"Final version, matching the published paper","Astrophys.J.679:1272-1287,2008",10.1086/533485,astro-ph,We show that the globular cluster mass funct...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...","Sun, 1 Apr 2007 10:20:37 GMT",2007-04-01 10:20:37
93,93,704.009,"HongSheng Zhao (SUPA, St Andrews)",Timing and Lensing of the Colliding Bullet Clu...,"5-pages, Physical Review D, rapid publication ...",NaN,NaN,astro-ph,We present semi-analytical constraint on the...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","Mon, 2 Apr 2007 18:30:16 GMT",2007-04-02 18:30:16
127,127,704.013,"P.A. Evans (1), A.P. Beardmore (1), K.L. Page ...",An online repository of Swift/XRT light curves...,"8 pages, 6 figures, Accepted for publication i...",NaN,10.1051/0004-6361:20077530,astro-ph,Context. Swift data are revolutionising our ...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","Mon, 2 Apr 2007 06:22:08 GMT",2007-04-02 06:22:08
132,132,704.013,"D. Lutz, E. Sturm, L.J. Tacconi, E. Valiante, ...",PAH emission and star formation in the host of...,Accepted for publication as an ApJ Letter. 14 ...,NaN,10.1086/518537,astro-ph,We report the first detection of the 6.2micr...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","Mon, 2 Apr 2007 06:51:10 GMT",2007-04-02 06:51:10
138,138,704.014,"B. Lanzoni (1,2), E. Dalessandro (1,2), F.R. F...",The Blue Straggler Population of the Globular ...,ApJ accepted,"Astrophys.J.663:267-276,2007",10.1086/518592,astro-ph,By combining high-resolution HST and wide-fi...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","Mon, 2 Apr 2007 08:19:37 GMT",2007-04-02 08:19:37
154,154,704.015,"Floris van der Tak (SRON Groningen), John Blac...",A computer program for fast non-LTE analysis o...,"Accepted by A&A; 18 A4 pages, 11 figures",NaN,10.1051/0004-6361:20066820,astro-ph,The large quantity and high quality of moder...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","Mon, 2 Apr 2007 09:38:00 GMT",2007-04-02 09:38:00


In [31]:
df_10.reset_index()

,index,Unnamed: 0,id,authors,title,comments,journal-ref,doi,categories,abstract,versions,v1_time,time
0,8,8,704.001,"Paul Harvey, Bruno Merin, Tracy L. Huard, Luis...","The Spitzer c2d Survey of Large, Nearby, Inste...",NaN,"Astrophys.J.663:1149-1173,2007",10.1086/518646,astro-ph,We discuss the results from the combined IRA...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","Mon, 2 Apr 2007 19:41:34 GMT",2007-04-02 19:41:34
1,16,16,704.002,"Nceba Mhlahlo, David H. Buckley, Vikram S. Dhi...",Spectroscopic Observations of the Intermediate...,"10 pages, 11 figures (figures 3, 4, 7 and 8 at...","Mon.Not.Roy.Astron.Soc.378:211-220,2007",10.1111/j.1365-2966.2007.11762.x,astro-ph,Results from spectroscopic observations of t...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...","Sat, 31 Mar 2007 07:38:48 GMT",2007-03-31 07:38:48
2,22,22,704.002,"M. A. Loukitcheva, S. K. Solanki and S. White",ALMA as the ideal probe of the solar chromosphere,"4 pages, 2 figures, to appear in the proceedin...","Astrophys.Space Sci.313:197-200,2008",10.1007/s10509-007-9626-1,astro-ph,"The very nature of the solar chromosphere, i...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...","Sat, 31 Mar 2007 11:42:13 GMT",2007-03-31 11:42:13
3,58,58,704.006,"T. G. Beatty, J. M. Fernandez, D. W. Latham, G...",The Mass and Radius of the Unseen M-Dwarf Comp...,"18 pages, 9 tables and 6 figures; accepted by ...","Astrophys.J.663:573-582,2007",10.1086/518413,astro-ph,We derive masses and radii for both componen...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...","Sat, 31 Mar 2007 22:16:50 GMT",2007-03-31 22:16:50
4,79,79,704.008,Dean E. McLaughlin and S. Michael Fall,Shaping the Globular Cluster Mass Function by ...,"Final version, matching the published paper","Astrophys.J.679:1272-1287,2008",10.1086/533485,astro-ph,We show that the globular cluster mass funct...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...","Sun, 1 Apr 2007 10:20:37 GMT",2007-04-01 10:20:37
5,93,93,704.009,"HongSheng Zhao (SUPA, St Andrews)",Timing and Lensing of the Colliding Bullet Clu...,"5-pages, Physical Review D, rapid publication ...",NaN,NaN,astro-ph,We present semi-analytical constraint on the...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","Mon, 2 Apr 2007 18:30:16 GMT",2007-04-02 18:30:16
6,127,127,704.013,"P.A. Evans (1), A.P. Beardmore (1), K.L. Page ...",An online repository of Swift/XRT light curves...,"8 pages, 6 figures, Accepted for publication i...",NaN,10.1051/0004-6361:20077530,astro-ph,Context. Swift data are revolutionising our ...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","Mon, 2 Apr 2007 06:22:08 GMT",2007-04-02 06:22:08
7,132,132,704.013,"D. Lutz, E. Sturm, L.J. Tacconi, E. Valiante, ...",PAH emission and star formation in the host of...,Accepted for publication as an ApJ Letter. 14 ...,NaN,10.1086/518537,astro-ph,We report the first detection of the 6.2micr...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","Mon, 2 Apr 2007 06:51:10 GMT",2007-04-02 06:51:10
8,138,138,704.014,"B. Lanzoni (1,2), E. Dalessandro (1,2), F.R. F...",The Blue Straggler Population of the Globular ...,ApJ accepted,"Astrophys.J.663:267-276,2007",10.1086/518592,astro-ph,By combining high-resolution HST and wide-fi...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","Mon, 2 Apr 2007 08:19:37 GMT",2007-04-02 08:19:37
9,154,154,704.015,"Floris van der Tak (SRON Groningen), John Blac...",A computer program for fast non-LTE analysis o...,"Accepted by A&A; 18 A4 pages, 11 figures",NaN,10.1051/0004-6361:20066820,astro-ph,The large quantity and high quality of moder...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","Mon, 2 Apr 2007 09:38:00 GMT",2007-04-02 09:38:00


In [37]:
# Plotting tools
import spacy
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [33]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [34]:
data_title = df_10.title.values.tolist()
# remove distracting characters
data_title = [re.sub("\'", "", sent) for sent in data_title]
data_title = [re.sub("$", "", sent) for sent in data_title]
data_title = [re.sub("{", "", sent) for sent in data_title]
data_title = [re.sub("}", "", sent) for sent in data_title]
data_title = [re.sub("}", "", sent) for sent in data_title]


In [35]:
# Remove new line characters
data_title = [re.sub('\s+', ' ', sent) for sent in data_title]
data_title = [re.sub('\n', ' ', sent) for sent in data_title]

In [38]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_title_words = list(sent_to_words(data_title))

print(data_title_words[:1])

[['the', 'spitzer', 'survey', 'of', 'large', 'nearby', 'insterstellar', 'clouds', 'ix', 'the', 'serpens', 'yso', 'population', 'as', 'observed', 'with', 'irac', 'and', 'mips']]


In [40]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
# Remove Stop Words
data_words_nostops = remove_stopwords(data_title_words)

In [41]:
nlp = spacy.load('en', disable=['tagger', 'textcat', 'parser', 'ner'])


In [44]:
def lemmatize_corpus(nlp, corpus):
    return [[w.lemma_ for w in nlp(' '.join(doc))] for doc in corpus]

# Do lemmatization keeping only noun, adj, vb, adv

data_lemmatized = lemmatize_corpus(nlp, data_words_nostops) #, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['spitzer', 'survey', 'large', 'nearby', 'insterstellar', 'cloud', 'ix', 'serpens', 'yso', 'population', 'observe', 'irac', 'mips']]


In [45]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [46]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes= 2,
                                           alpha='auto',
                                           per_word_topics=True)

In [47]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17     0.319319  0.323203       1        1  10.306474
14     0.191109 -0.282036       2        1   8.725823
19     0.173124 -0.194035       3        1   8.148026
10     0.095297 -0.048345       4        1   7.693134
6      0.046094  0.024445       5        1   5.749478
9      0.002738 -0.001135       6        1   5.037675
11    -0.017656  0.004596       7        1   4.837683
16    -0.019155  0.004921       8        1   4.834325
3     -0.043186  0.010407       9        1   4.446005
5     -0.027076  0.006823      10        1   4.420799
12    -0.058736  0.013253      11        1   4.217747
7     -0.051313  0.011888      12        1   4.133214
2     -0.054097  0.012435      13        1   4.120197
0     -0.048295  0.011274      14        1   3.895169
13    -0.068256  0.014936      15        1   3.841446
4     -0.065182  0.014262      16        1   3.760587
8     -0.084622  0.017204      17        1   3.294353
1     -0.093285  0.018284      18        1   3.199805
18    -0.095539  0.018509      19        1   2.829344
15    -0.101282  0.019111      20        1   2.508719, topic_info=      Category          Freq           Term         Total  loglift  logprob
term                                                                       
122    Default  16014.000000         galaxy  16014.000000  30.0000  30.0000
64     Default  13486.000000           star  13486.000000  29.0000  29.0000
108    Default  11636.000000            ray  11636.000000  28.0000  28.0000
36     Default   7603.000000        cluster   7603.000000  27.0000  27.0000
192    Default   5273.000000          model   5273.000000  26.0000  26.0000
11     Default   4826.000000         survey   4826.000000  25.0000  25.0000
31     Default   4546.000000           mass   4546.000000  24.0000  24.0000
121    Default   3805.000000           disk   3805.000000  23.0000  23.0000
203    Default   3546.000000      evolution   3546.000000  22.0000  22.0000
16     Default   5295.000000    observation   5295.000000  21.0000  21.0000
59     Default   4210.000000       emission   4210.000000  20.0000  20.0000
60     Default   4320.000000      formation   4320.000000  19.0000  19.0000
80     Default   3651.000000          field   3651.000000  18.0000  18.0000
27     Default   3144.000000          dwarf   3144.000000  17.0000  17.0000
133    Default   3233.000000      supernova   3233.000000  16.0000  16.0000
25     Default   3456.000000         binary   3456.000000  15.0000  15.0000
158    Default   3214.000000            ngc   3214.000000  14.0000  14.0000
42     Default   4096.000000        stellar   4096.000000  13.0000  13.0000
104    Default   4462.000000           high   4462.000000  12.0000  12.0000
359    Default   3382.000000       galactic   3382.000000  11.0000  11.0000
130    Default   3956.000000          radio   3956.000000  10.0000  10.0000
24     Default   2895.000000          solar   2895.000000   9.0000   9.0000
143    Default   2980.000000           dark   2980.000000   8.0000   8.0000
30     Default   3077.000000           line   3077.000000   7.0000   7.0000
429    Default   2713.000000      structure   2713.000000   6.0000   6.0000
222    Default   2761.000000           type   2761.000000   5.0000   5.0000
103    Default   3772.000000          gamma   3772.000000   4.0000   4.0000
225    Default   2904.000000       infrared   2904.000000   3.0000   3.0000
148    Default   2730.000000            new   2730.000000   2.0000   2.0000
85     Default   2664.000000       magnetic   2664.000000   1.0000   1.0000
...        ...           ...            ...           ...      ...      ...
1772   Topic20      0.082081    gaussianity      1.756766   0.6219 -11.6569
687    Topic20      0.082078          event      1.756846   0.6218 -11.6570
1066   Topic20      0.082077           warm      1.756851   0.6218 -11.6570
4210   Topic20      0.082071     

In [48]:
# Build LDA model
lda_model_10 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes= 2,
                                           alpha='auto',
                                           per_word_topics=True)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_10, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.454102  0.006214       1        1  16.471657
3      0.029379  0.184524       2        1  11.808816
0      0.036641 -0.427434       3        1  11.460259
1      0.055076  0.055470       4        1  10.681016
9      0.054870  0.048060       5        1  10.327236
8      0.053962  0.030112       6        1   8.399590
4      0.057748  0.019343       7        1   8.321076
7      0.056371  0.026951       8        1   7.750237
5      0.051272  0.030329       9        1   7.706993
6      0.058782  0.026430      10        1   7.073111, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
122   Default  8345.000000         galaxy  8345.000000  30.0000  30.0000
108   Default  6513.000000            ray  6513.000000  29.0000  29.0000
64    Default  7628.000000           star  7628.000000  28.0000  28.0000
36    Default  4512.000000        cluster  4512.000000  27.0000  27.0000
11    Default  2723.000000         survey  2723.000000  26.0000  26.0000
16    Default  3147.000000    observation  3147.000000  25.0000  25.0000
31    Default  2655.000000           mass  2655.000000  24.0000  24.0000
192   Default  2888.000000          model  2888.000000  23.0000  23.0000
59    Default  2558.000000       emission  2558.000000  22.0000  22.0000
104   Default  2497.000000           high  2497.000000  21.0000  21.0000
25    Default  2011.000000         binary  2011.000000  20.0000  20.0000
80    Default  2218.000000          field  2218.000000  19.0000  19.0000
158   Default  2003.000000            ngc  2003.000000  18.0000  18.0000
121   Default  2186.000000           disk  2186.000000  17.0000  17.0000
130   Default  2306.000000          radio  2306.000000  16.0000  16.0000
133   Default  1871.000000      supernova  1871.000000  15.0000  15.0000
103   Default  2112.000000          gamma  2112.000000  14.0000  14.0000
359   Default  1981.000000       galactic  1981.000000  13.0000  13.0000
42    Default  2430.000000        stellar  2430.000000  12.0000  12.0000
109   Default  1693.000000         source  1693.000000  11.0000  11.0000
222   Default  1700.000000           type  1700.000000  10.0000  10.0000
27    Default  1696.000000          dwarf  1696.000000   9.0000   9.0000
30    Default  1830.000000           line  1830.000000   8.0000   8.0000
24    Default  1816.000000          solar  1816.000000   7.0000   7.0000
60    Default  2251.000000      formation  2251.000000   6.0000   6.0000
429   Default  1566.000000      structure  1566.000000   5.0000   5.0000
102   Default  1848.000000         energy  1848.000000   4.0000   4.0000
225   Default  1764.000000       infrared  1764.000000   3.0000   3.0000
74    Default  1732.000000       spectrum  1732.000000   2.0000   2.0000
85    Default  1618.000000       magnetic  1618.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
579   Topic10  1291.936768         cosmic  1292.849609   2.6482  -3.0295
672   Topic10   976.112732   spectroscopy   977.025574   2.6479  -3.3098
67    Topic10   937.998535       analysis   938.911377   2.6479  -3.3497
543   Topic10   936.807068         quasar   937.719910   2.6479  -3.3509
140   Topic10   833.133972           wind   834.046814   2.6478  -3.4682
138   Topic10   817.321838        neutron   818.234680   2.6478  -3.4874
396   Topic10   794.182434       universe   795.095337   2.6477  -3.5161
785   Topic10   581.638123           rate   582.551025   2.6473  -3.8276
237   Topic10   546.267334     absorption   547.180176   2.6472  -3.8903
484   Topic10   532.694641     background   533.607422   2.6472  -3.9155
313   Topic10   476.629578    oscillation   477.542450   2.6470  -4.0267
812   Topic10   432.629913           flow   433.542725   2.6468  -4.1235
240   Topic10   371.1765

In [50]:
start_date = '2004-08-01'
end_date = '2008-11-30'
mask = (df_astro['time'] > start_date) & (df_astro['time'] <= end_date)
df_10 = df_astro.loc[mask]
df_10.reset_index()

data_title = df_10.title.values.tolist()
# remove distracting characters
data_title = [re.sub("\'", "", sent) for sent in data_title]
data_title = [re.sub("$", "", sent) for sent in data_title]
data_title = [re.sub("{", "", sent) for sent in data_title]
data_title = [re.sub("}", "", sent) for sent in data_title]
data_title = [re.sub("}", "", sent) for sent in data_title]
# Remove new line characters
data_title = [re.sub('\s+', ' ', sent) for sent in data_title]
data_title = [re.sub('\n', ' ', sent) for sent in data_title]
data_title_words = list(sent_to_words(data_title))

data_words_nostops = remove_stopwords(data_title_words)
data_lemmatized = lemmatize_corpus(nlp, data_words_nostops) #, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# Build LDA model
lda_model_10 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes= 2,
                                           alpha='auto',
                                           per_word_topics=True)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_10, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10     0.135178  0.159887       1        1  8.059493
11     0.169849  0.293909       2        1  8.040619
8      0.101299  0.068190       3        1  7.055681
3      0.083543  0.023029       4        1  6.942800
4      0.036123 -0.035916       5        1  5.887264
1      0.038925 -0.032546       6        1  5.708250
17     0.059685 -0.010463       7        1  5.553529
19    -0.316215  0.212566       8        1  5.493103
9      0.032471 -0.023916       9        1  5.159248
2      0.012283 -0.058832      10        1  4.997860
14     0.029603 -0.042387      11        1  4.913394
5      0.002982 -0.066434      12        1  4.421853
0     -0.005638 -0.072662      13        1  4.392728
6     -0.012362 -0.076911      14        1  4.114279
16    -0.013842 -0.077874      15        1  3.847216
15    -0.013438 -0.077697      16        1  3.586533
7     -0.026601 -0.084729      17        1  3.381649
12    -0.025106 -0.083884      18        1  3.355030
18    -0.244366  0.077209      19        1  2.614027
13    -0.044372 -0.090538      20        1  2.475446, topic_info=     Category          Freq           Term         Total  loglift  logprob
term                                                                      
122   Default  11834.000000         galaxy  11834.000000  30.0000  30.0000
64    Default  10664.000000           star  10664.000000  29.0000  29.0000
108   Default   8397.000000            ray   8397.000000  28.0000  28.0000
36    Default   5659.000000        cluster   5659.000000  27.0000  27.0000
16    Default   3979.000000    observation   3979.000000  26.0000  26.0000
192   Default   3730.000000          model   3730.000000  25.0000  25.0000
11    Default   3867.000000         survey   3867.000000  24.0000  24.0000
104   Default   3586.000000           high   3586.000000  23.0000  23.0000
31    Default   3242.000000           mass   3242.000000  22.0000  22.0000
42    Default   3103.000000        stellar   3103.000000  21.0000  21.0000
80    Default   3026.000000          field   3026.000000  20.0000  20.0000
59    Default   3267.000000       emission   3267.000000  19.0000  19.0000
203   Default   2814.000000      evolution   2814.000000  18.0000  18.0000
158   Default   2643.000000            ngc   2643.000000  17.0000  17.0000
359   Default   2787.000000       galactic   2787.000000  16.0000  16.0000
60    Default   3112.000000      formation   3112.000000  15.0000  15.0000
30    Default   2399.000000           line   2399.000000  14.0000  14.0000
133   Default   2501.000000      supernova   2501.000000  13.0000  13.0000
130   Default   2926.000000          radio   2926.000000  12.0000  12.0000
121   Default   3034.000000           disk   3034.000000  11.0000  11.0000
102   Default   2558.000000         energy   2558.000000  10.0000  10.0000
74    Default   2342.000000       spectrum   2342.000000   9.0000   9.0000
24    Default   2240.000000          solar   2240.000000   8.0000   8.0000
27    Default   2180.000000          dwarf   2180.000000   7.0000   7.0000
25    Default   2496.000000         binary   2496.000000   6.0000   6.0000
103   Default   2766.000000          gamma   2766.000000   5.0000   5.0000
109   Default   2363.000000         source   2363.000000   4.0000   4.0000
148   Default   2153.000000            new   2153.000000   3.0000   3.0000
85    Default   2240.000000       magnetic   2240.000000   2.0000   2.0000
222   Default   2397.000000           type   2397.000000   1.0000   1.0000
...       ...           ...            ...           ...      ...      ...
256   Topic20    410.448975     turbulence    412.120056   3.6947  -2.8706
163   Topic20    403.798035           main    405.469116   3.6946  -2.8870
421   Topic20    332.241302       outburst    333.912384   3.6937  -3.0820
422   Topic20    314.246277      transient    315.917358   3.6934  -3.1377
387   Topic20   